In [ ]:
#Required Libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras_tuner import RandomSearch

In [ ]:
#Read the data
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
val_df=pd.read_csv('dev.csv')

In [4]:
#Preprocess the data

def preprocess(df):
    #Tokenization
    tokenizer=Tokenizer(num_words=5000, split="")
    tokenizer.fit_on_texts(df['source'].values)
    #Turning the tokenized text into sequences
    X=tokenizer.texts_to_sequences (df['source'].values)
    X=pad_sequences(X) # padding our text vector so they all have the same langth
    #Label Encoding
    le=LabelEncoder()
    Y=le.fit_transform(df['target'])
    Y=to_categorical (Y) #one-hot encoding
    return X, Y

In [ ]:
X_train, Y_train= preprocess(train_df)
X_test, Y_test =preprocess(test_df)
x_val, Y_val =preprocess(val_df)

In [5]:

#Model Definition
def build_model(hp):
    model=Sequential()
    model.add(Embedding(5000, hp.Int("embedding_vector_length", min_value=32, max_value=512, step=32), input_length = X_train.shape[1]))
    model.add(LSTM(hp.Int('1stm_units', min_value=32, max_value=512, step=32), dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(hp.Int('dense_units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dense(Y_train.shape[1], activation='softmax')) # output layer size should match the number of classes
    model.compile(loss='categorical_crossentropy', optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), metrics=['accuracy'])
    return model

In [ ]:
#Hyperparameter Tuning
tuner=RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='brand_identification')

tuner.search_space_summary()
tuner.search(X_train, Y_train, epochs=5, validation_data=(x_val,Y_val))

In [ ]:
#Get the optimal hyperparameters

best_hps=tuner.get_best_hyperparameters (num_trials=1)[0]

print(f"The hyperparameter search is complete. The optimal number of units in the LSTM layer is {best_hps.get('lstm_units')} and the optimal length of the embedding vector is {best_hps.get('embedding_vector_length')}.")

#Training the model with the optimal hyperparameters

model=tuner.hypermodel.build(best_hps) 
model.fit(X_train, Y_train, epochs=10, batch_size=64, verbose=2, validation_data=(x_val, Y_val))

#Saving the model

model.save('my_model.h5')